# 3. Download and prepare country stats

Downloads statistics from google cloud storage and combines them with FAO STAT data
on emissions land use and forests. Creates a single csv file with all data.


In [5]:
import os

import pandas as pd

# Set up credentials
GEE_JSON = os.getenv("GEE_JSON")
PROJECT = os.getenv("PROJECT")
EXPORT_BUCKET = os.getenv("GEE_BUCKET")
EXPORT_PREFIX = "landscape_indicators_20231023"

DATA_FOLDER = "../data/raw"

In [6]:
drop_columns = [
    "system:index",
    "ADM0_NAME",
    "DISP_AREA",
    "EXP0_YEAR",
    "STATUS",
    "STR0_YEAR",
    "Shape_Area",
    "Shape_Leng",
    ".geo",
]

country_stats = pd.read_csv(f"gs://{EXPORT_BUCKET}/{EXPORT_PREFIX}/country_stats_100m.csv")
country_kernel_stats = pd.read_csv(
    f"gs://{EXPORT_BUCKET}/{EXPORT_PREFIX}/sluc_kernel_country_stats_1000m.csv"
)
country_stats.set_index("ADM0_CODE", inplace=True)
country_stats.drop(columns=drop_columns, inplace=True)
country_kernel_stats.set_index("ADM0_CODE", inplace=True)
country_kernel_stats.drop(columns=drop_columns, inplace=True)
country_stats = country_stats.join(country_kernel_stats, rsuffix="2")

print(country_stats.columns)
country_stats.head()

Index(['const', 'cropland', 'deforest', 'deforest_carbon',
       'dluc_cropland_deforest', 'dluc_cropland_deforest_carbon',
       'dluc_cropland_nattreeloss', 'dluc_cropland_treeloss',
       'dluc_nonnat_deforest', 'dluc_nonnat_deforest_carbon',
       'dluc_nonnat_nattreeloss', 'dluc_nonnat_treeloss', 'nat_treeloss',
       'nonnat', 'treeloss', 'const2',
       'cropland_deforest_by_human_lu_0km_1000m',
       'cropland_deforest_by_human_lu_10km_1000m',
       'cropland_deforest_by_human_lu_25km_1000m',
       'cropland_deforest_by_human_lu_50km_1000m',
       'cropland_deforest_by_human_lu_5km_1000m',
       'cropland_deforest_carbon_by_human_lu_0km_1000m',
       'cropland_deforest_carbon_by_human_lu_10km_1000m',
       'cropland_deforest_carbon_by_human_lu_25km_1000m',
       'cropland_deforest_carbon_by_human_lu_50km_1000m',
       'cropland_deforest_carbon_by_human_lu_5km_1000m',
       'cropland_natural_crop_net_conversion_by_human_lu_0km_1000m',
       'cropland_natural_cro

,const,cropland,deforest,deforest_carbon,dluc_cropland_deforest,dluc_cropland_deforest_carbon,dluc_cropland_nattreeloss,dluc_cropland_treeloss,dluc_nonnat_deforest,dluc_nonnat_deforest_carbon,...,nonnat_natural_crop_net_conversion_by_human_lu_0km_1000m,nonnat_natural_crop_net_conversion_by_human_lu_10km_1000m,nonnat_natural_crop_net_conversion_by_human_lu_25km_1000m,nonnat_natural_crop_net_conversion_by_human_lu_50km_1000m,nonnat_natural_crop_net_conversion_by_human_lu_5km_1000m,nonnat_tree_loss_by_human_lu_0km_1000m,nonnat_tree_loss_by_human_lu_10km_1000m,nonnat_tree_loss_by_human_lu_25km_1000m,nonnat_tree_loss_by_human_lu_50km_1000m,nonnat_tree_loss_by_human_lu_5km_1000m
ADM0_CODE,,,,,,,,,,,,,,,,,,,,,
1,6.418374e+05,28103.062837,0.226144,3.596515,0.061129,0.155029,0.004723,0.067989,0.104009,0.788664,...,226.009332,252.812782,249.434493,258.646991,247.799409,1.178819,0.926596,0.962920,1.304262,0.945994
2,3.066542e+04,0.000000,0.000502,0.001079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000083,0.000060,0.000147,0.000103
3,2.868165e+04,2001.537786,5.714428,305.953310,0.065749,2.348173,0.028483,0.085550,0.623648,26.557240,...,16.143611,5.416760,3.335665,2.074649,7.490282,18.090614,23.885509,25.292438,24.989412,21.702992
4,2.308743e+06,52583.702468,23.798736,945.174251,2.944335,81.572399,0.761052,3.774155,6.236238,190.385781,...,501.831544,432.424044,430.955615,409.939208,435.046049,61.456965,110.267648,128.000490,119.097907,94.071457
5,1.999620e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
# Load FAO data

country_df = pd.read_csv(f"{DATA_FOLDER}/country-codes.csv")
fao_forest_data = pd.read_csv(
    f"{DATA_FOLDER}/Emissions_Land_Use_Forests_E_All_Data_(Normalized).csv", encoding="latin-1"
)
fao_landuse_data = pd.read_csv(
    f"{DATA_FOLDER}/Inputs_LandUse_E_All_Data_(Normalized).csv", encoding="latin-1"
)

fao_forest_data = fao_forest_data[
    ((fao_forest_data["Year"] <= 2022) & (fao_forest_data["Year"] > 2002))
]
fao_forest_data["M49"] = fao_forest_data["Area Code (M49)"].apply(lambda x: x[1:]).astype(float)
fao_landuse_data = fao_landuse_data[
    ((fao_landuse_data["Year"] <= 2022) & (fao_landuse_data["Year"] > 2002))
]
fao_landuse_data["M49"] = fao_landuse_data["Area Code (M49)"].apply(lambda x: x[1:]).astype(float)

fao_area = (
    fao_forest_data[
        (
            (fao_forest_data["Source"] == "FAO TIER 1")
            & (fao_forest_data["Item"] == "Net Forest conversion")
            & (fao_forest_data["Element"] == "Area")
        )
    ]
    .groupby("M49")["Value"]
    .mean()
    * 10
)  # convert to km2
fao_area.name = "fao_forestloss"

fao_emissions = (
    fao_forest_data[
        (
            (fao_forest_data["Source"] == "FAO TIER 1")
            & (fao_forest_data["Item"] == "Net Forest conversion")
            & (fao_forest_data["Element"] == "Net emissions/removals (CO2) (Forest land)")
        )
    ]
    .groupby("M49")["Value"]
    .mean()
    * 1000
    # kT to tons
)
fao_emissions.name = "fao_emissions"

fao_cropland = (
    fao_landuse_data[((fao_landuse_data["Item"] == "Cropland"))].groupby("M49")["Value"].mean() * 10
)  # convert to km2
fao_cropland.name = "fao_cropland"

fao_agland = (
    fao_landuse_data[((fao_landuse_data["Item"] == "Agricultural land"))]
    .groupby("M49")["Value"]
    .mean()
    * 10
)  # convert to km2
fao_agland.name = "fao_agland"

all_fao = pd.concat([fao_area, fao_emissions, fao_cropland, fao_agland], axis=1)
all_fao.head()

,fao_forestloss,fao_emissions,fao_cropland,fao_agland
M49,,,,
1.0,111438.292111,3.376843e+09,3.851482e+06,2.396757e+07
2.0,38708.883556,1.048339e+09,6.722308e+05,5.677998e+06
4.0,0.000000,0.000000e+00,1.992586e+04,1.901139e+05
5.0,44025.451111,1.616329e+09,3.243051e+05,2.685559e+06
8.0,0.000000,0.000000e+00,1.949781e+03,6.024428e+03


In [8]:
c = country_df.drop(211)
c.GAUL = c.GAUL.astype(float)
c = c.filter(["GAUL", "CLDR display name", "Sub-region Name", "M49", "ISO3166-1-Alpha-3"])
c.rename(columns={"CLDR display name": "country_name", "Sub-region Name": "region"}, inplace=True)

all_stats = (
    country_stats.groupby("ADM0_CODE").sum().join(c.set_index("GAUL")).join(all_fao, on="M49")
)

all_stats["deforest_carbon"] *= 100 * 3.66  # ha->km2 and tC -> tCO2e
all_stats["nonnat_deforest_carbon_by_human_lu_0km_1000m"] *= 100  # ha->km2
all_stats["nonnat_deforest_carbon_by_human_lu_5km_1000m"] *= 100  # ha->km2
all_stats["nonnat_deforest_carbon_by_human_lu_10km_1000m"] *= 100  # ha->km2
all_stats["nonnat_deforest_carbon_by_human_lu_25km_1000m"] *= 100  # ha->km2
all_stats["nonnat_deforest_carbon_by_human_lu_50km_1000m"] *= 100  # ha->km2
all_stats["cropland_deforest_carbon_by_human_lu_0km_1000m"] *= 100  # ha->km2
all_stats["cropland_deforest_carbon_by_human_lu_5km_1000m"] *= 100  # ha->km2
all_stats["cropland_deforest_carbon_by_human_lu_10km_1000m"] *= 100  # ha->km2
all_stats["cropland_deforest_carbon_by_human_lu_25km_1000m"] *= 100  # ha->km2
all_stats["cropland_deforest_carbon_by_human_lu_50km_1000m"] *= 100  # ha->km2

# annualize
all_stats["sluc_treeloss_nonnat_ha"] = all_stats["treeloss"] / all_stats["nonnat"]
all_stats["sluc_treeloss_cropland_ha"] = all_stats["treeloss"] / all_stats["cropland"]
all_stats["sluc_treeloss_cropland"] = all_stats["cropland"] * all_stats["sluc_treeloss_nonnat_ha"]
all_stats["sluc_nat_treeloss_nonnat_ha"] = all_stats["nat_treeloss"] / all_stats["nonnat"]
all_stats["sluc_nat_treeloss_cropland_ha"] = all_stats["nat_treeloss"] / all_stats["cropland"]
all_stats["sluc_nat_treeloss_cropland"] = (
    all_stats["cropland"] * all_stats["sluc_nat_treeloss_nonnat_ha"]
)
all_stats["sluc_deforest_nonnat_ha"] = all_stats["deforest"] / all_stats["nonnat"]
all_stats["sluc_deforest_cropland_ha"] = all_stats["deforest"] / all_stats["cropland"]
all_stats["sluc_deforest_cropland"] = all_stats["cropland"] * all_stats["sluc_deforest_nonnat_ha"]
all_stats["sluc_deforest_carbon_nonnat_ha"] = all_stats["deforest_carbon"] / all_stats["nonnat"]
all_stats["sluc_deforest_carbon_cropland_ha"] = all_stats["deforest_carbon"] / all_stats["cropland"]
all_stats["sluc_deforest_carbon_cropland"] = (
    all_stats["cropland"] * all_stats["sluc_deforest_carbon_nonnat_ha"]
)
all_stats["sluc_fao_agland_ha"] = all_stats["fao_forestloss"] / all_stats["fao_agland"]
all_stats["sluc_fao_cropland_ha"] = all_stats["fao_forestloss"] / all_stats["fao_cropland"]
all_stats["sluc_fao_cropland"] = all_stats["fao_cropland"] * all_stats["sluc_fao_agland_ha"]
all_stats["sluc_emissions_fao_agland_ha"] = all_stats["fao_emissions"] / all_stats["fao_agland"]
all_stats["sluc_emissions_fao_cropland_ha"] = all_stats["fao_emissions"] / all_stats["fao_cropland"]
all_stats["sluc_emissions_fao_cropland"] = (
    all_stats["fao_cropland"] * all_stats["sluc_emissions_fao_agland_ha"]
)

print(all_stats.columns)
all_stats.to_csv("../data/processed/all_country_stats.csv")
all_stats.head()

Index(['const', 'cropland', 'deforest', 'deforest_carbon',
       'dluc_cropland_deforest', 'dluc_cropland_deforest_carbon',
       'dluc_cropland_nattreeloss', 'dluc_cropland_treeloss',
       'dluc_nonnat_deforest', 'dluc_nonnat_deforest_carbon',
       'dluc_nonnat_nattreeloss', 'dluc_nonnat_treeloss', 'nat_treeloss',
       'nonnat', 'treeloss', 'const2',
       'cropland_deforest_by_human_lu_0km_1000m',
       'cropland_deforest_by_human_lu_10km_1000m',
       'cropland_deforest_by_human_lu_25km_1000m',
       'cropland_deforest_by_human_lu_50km_1000m',
       'cropland_deforest_by_human_lu_5km_1000m',
       'cropland_deforest_carbon_by_human_lu_0km_1000m',
       'cropland_deforest_carbon_by_human_lu_10km_1000m',
       'cropland_deforest_carbon_by_human_lu_25km_1000m',
       'cropland_deforest_carbon_by_human_lu_50km_1000m',
       'cropland_deforest_carbon_by_human_lu_5km_1000m',
       'cropland_natural_crop_net_conversion_by_human_lu_0km_1000m',
       'cropland_natural_cro

,const,cropland,deforest,deforest_carbon,dluc_cropland_deforest,dluc_cropland_deforest_carbon,dluc_cropland_nattreeloss,dluc_cropland_treeloss,dluc_nonnat_deforest,dluc_nonnat_deforest_carbon,...,sluc_deforest_cropland,sluc_deforest_carbon_nonnat_ha,sluc_deforest_carbon_cropland_ha,sluc_deforest_carbon_cropland,sluc_fao_agland_ha,sluc_fao_cropland_ha,sluc_fao_cropland,sluc_emissions_fao_agland_ha,sluc_emissions_fao_cropland_ha,sluc_emissions_fao_cropland
ADM0_CODE,,,,,,,,,,,,,,,,,,,,,
1.0,6.418374e+05,28103.062837,0.226144,1316.324436,0.061129,0.155029,0.004723,0.067989,0.104009,0.788664,...,0.073256,0.015173,0.046839,426.405461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2.0,3.066542e+04,0.000000,0.000502,0.394895,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.205093,inf,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
3.0,2.868165e+04,2001.537786,5.714428,111978.911554,0.065749,2.348173,0.028483,0.085550,0.623648,26.557240,...,1.740409,17.039273,55.946439,34104.748068,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4.0,2.308743e+06,52583.702468,23.798736,345933.775916,2.944335,81.572399,0.761052,3.774155,6.236238,190.385781,...,12.861472,3.555319,6.578726,186951.845670,0.000178,0.001737,3.751314,1.230603,12.028345,25981.181100
5.0,1.999620e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,NaN,0.000000,0.003802,0.001036,1.101049,108.777003,29.638200,31499.188021
